# core

> Main(/all) routines for `ipyfernel`

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
#| hide
from jupyter_client.manager import KernelManager
from IPython.display import display, Image
import base64
from pathlib import Path
from IPython.core.magic import register_line_magic, register_line_cell_magic

## Remote Kernel Backend

In [ ]:
#| export
#| notest
def set_ssh_config(
    port:int,                       # Port number on proxy server (e.g. bore.pub)
    user:str="",                    # Username on remote system.
    alias="remote_server_sshpyk",   # Default alias for `sshpyk`, leave it alone
    proxyname="bore.pub",           # Have tested this with bore
    config_path="~/.ssh/config",    # Shouldn't need to change this.
    ):
    config_path = Path(config_path).expanduser()
    if not config_path.exists(): config_path.touch()
    text = config_path.read_text()
    if f"Host {alias}" not in text: 
        assert user != "", "Must specify username when creating ~/.ssh/config info"
        block = f"""
Host {alias}
    HostName {proxyname}
    Port {port}
    User {user}
    BatchMode yes
    ControlMaster auto
    ControlPath ~/.ssh/sshpyk_%r@%h_%p
    ControlPersist 10m
    StrictHostKeyChecking no
    UserKnownHostsFile /dev/null
"""
        config_path.write_text(text + block)
    else:
        lines = text.splitlines()
        in_target_block = False
        for i, line in enumerate(lines):
            if line.startswith("Host "):
                in_target_block = (line == f"Host {alias}")
            elif in_target_block and line.strip().startswith("Port "):
                lines[i] = f"    Port {port}"
            elif proxyname and in_target_block and line.strip().startswith("HostName "):
                lines[i] = f"    HostName {proxyname}"
            elif user and in_target_block and line.strip().startswith("User "):
                lines[i] = f"    User {user}"
        config_path.write_text("\n".join(lines) + "\n")
    print(f'{config_path} file updated.') 

In [ ]:
port = 40308
set_ssh_config(port) 

/app/data/.ssh/config file updated.


In [ ]:
#| export
#| notest
_ipf_km, _ipf_kc = None, None            # "ipf" = "ipyfernel" ;-) 
def ipf_startup(kernel_name="ipyf_remote_kernel"):  
    "Start up the remote kernel"
    global _ipf_km, _ipf_kc 
    if _ipf_km is None and _ipf_kc is None: #only do this at startup
        _ipf_km = KernelManager(kernel_name=kernel_name)
        _ipf_km.start_kernel()
        _ipf_kc = _ipf_km.client()
        _ipf_kc.start_channels()
        _ipf_kc.wait_for_ready(timeout=30)
        print("Success: remote kernel started")
    else: 
        print("ipf_startup: already running")

In [ ]:
ipf_startup()

Success: remote kernel started


In [ ]:
#| export
#| notest
def _output_hook(
    msg,   #  Message obtained from remote execution
    ):
    "How to handle output from the remote kernel."
    mt = msg["msg_type"]
    content = msg.get("content", {})
    if mt == "stream":
        print(content["text"], end="", flush=True)
    elif mt == "error":
        print('\n'.join(content.get("traceback", [])))
    elif mt in ("display_data", "update_display_data"):
        data = content.get("data", {})
        if "image/png" in data:
            display(Image(base64.b64decode(data["image/png"])))
        elif "text/plain" in data:
            print(data["text/plain"])

In [ ]:
#| export
#| notest
def ipf_exec(
    code:str,           # Code to be executed
    verbose=False,      # Return details about remote execution.
    ):
    "Execute code on the remote kernel." 
    assert _ipf_kc is not None, "ipf_exec: need to run ipf_startup() first"
    result = _ipf_kc.execute_interactive(code=code, output_hook=_output_hook)
    _ipf_kc.last_result = result  # stash it for optional inspection later
    if verbose: return result

In [ ]:
code = """
import platform 
print(platform.system())
"""
ipf_exec(code)

Darwin


In [ ]:
#| export
#| notest
def ipf_shutdown(verbose=True):
    "Terminates the remote kernel"
    global _ipf_km, _ipf_kc
    if verbose: print("Shutting down remote kernel") # Note: Could make say if remote kernel is not even running.
    try:
        if _ipf_kc is not None: _ipf_kc.stop_channels()
        if _ipf_km is not None: _ipf_km.shutdown_kernel(now=True)  # 'now=True' forces immediate shutdown
    except: pass  # Don't hang on errors
    _ipf_km, _ipf_kc = None, None

In [ ]:
ipf_shutdown()

Shutting down remote kernel


# iPython Magics

In [ ]:
#| export
#| notest
def _execute_remotely(lines:list[str]):
    "Take commands from magics and send to ipf_exec"
    code = ''.join(lines)
    if 'get_ipython()' in code: return lines  # let solveit internals pass through
    # Make sure our magics execute locally
    if code.strip().startswith(('%local', '%%local', '%unset_remote', '%resume_remote', '%set_remote', '%set_sticky','%unset_sticky')):
        return lines
    return [f"ipf_exec({repr(code)})\n"]

In [ ]:
#| export
#| notest
@register_line_magic
def set_remote(line:str):
    """Setup connection to remote server, start remote server, and enable 'sticky' remote execution of code cells (even without magics).
    usage: %set_remote <port> [user]"""
    parts = line.split()
    port = int(parts[0]) if parts else 65445
    user = parts[1] if len(parts) > 1 else ""
    set_ssh_config(port, user=user) 
    try: 
        ipf_startup()
    except Exception as e: 
        print(f"Error starting up remote kernel: {e}") 
        return 

In [ ]:
%set_remote {port}

/app/data/.ssh/config file updated.


Success: remote kernel started


In [ ]:
#| export
@register_line_cell_magic
def remote(line, cell=None):
    "remote exeuction: works as %remote and as %%remote" 
    ipf_exec(cell if cell else line)

In [ ]:
%%remote 
import socket 
hostname = socket.gethostname()   # let's make sure we're running remotely
print("Hello from",hostname) 

Hello from Chonk


In [ ]:
#| export
@register_line_cell_magic
def local(line, cell=None):
    "local execution: works as %local and as %%local"
    get_ipython().run_cell(cell if cell else line) 

In [ ]:
%%local 
import socket 
hostname = socket.gethostname()   # let's make sure we're running remotely
print("Hello from",hostname) 

Hello from 909e93a3a71e


In [ ]:
#| export
@register_line_magic
def unset_remote(line):
    "shutdown remote server"
    unset_sticky('')  # get rid of any input transformers (see below) 
    ipf_shutdown()

## 'Sticky'/'Seamless' Remote Excution 

via Input Transformers.  These can make cells set execute remotely by default.

**WARNINGS**: 
1. Solve it is not intended to work with people modifying input transformers So be wary. Nevertheless, this seems to work.
2. If they're commands that you definitely want to execute locally, maybe run `%unset_sticky` just to be sure first.


In [ ]:
#| export
gip = get_ipython()

@register_line_magic
def set_sticky(line):
    """Adds execute remotely to input transformer list (if it's not there already).  WARNING: Can break solveit"""
    assert _ipf_kc is not None, "Need an active remote kernel connection" 
    for f in gip.input_transformers_cleanup[:]:   # gaurd against appending twice
        if getattr(f, '__name__', '') == '_execute_remotely':
            print("Already executing remotely") 
            return 
    gip.input_transformers_cleanup.append(_execute_remotely)
    print('Code cells will now execute remotely.')

In [ ]:
#| export
@register_line_magic
def unset_sticky(line):
    for f in gip.input_transformers_cleanup[:]:  
        if getattr(f, '__name__', '') == '_execute_remotely':
            gip.input_transformers_cleanup.remove(f)
    print("Code cells will now run locally.") 



In [ ]:
%unset_remote 
%set_remote {port}
%set_sticky

hostname =  909e93a3a71e
Code cells will now run locally.
Shutting down remote kernel


/app/data/.ssh/config file updated.


Success: remote kernel started
Code cells will now execute remotely.


In [ ]:
import socket 
hostname = socket.gethostname()
print("Hello from",hostname) 

Hello from Chonk


In [ ]:
%unset_sticky

hostname =  909e93a3a71e
Code cells will now run locally.


In [ ]:
import socket 
hostname = socket.gethostname()
print("Hello from",hostname) 

Hello from 909e93a3a71e


Question. I want to show some demos of progress bars and making plots remotely. Should I include them here or should I make a separate notebook called demo? Just not sure that demo actually needs to be really part of the repo, or not? There wouldn't be any exported code, but maybe it'd be more like a tutorial kind of thing. So, yeah, the question is: should I include the demoing in this notebook or should I make it a separate notebook?

I guess for users that don't really care about all this code and they just want to see what's the basic usage, maybe demo would be a good thing. And then I just set it to like export none or export false or something.

?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good question! Here's my take:

**Separate demo notebook** makes sense here for a few reasons:

1. **Audience separation** — `core.nbs` is for people who want to understand/modify the internals. A `demo.nbs` or `tutorial.nbs` is for people who just want to *use* it and see what's possible.

2. **Cleaner core** — Your core notebook is already nicely focused on the implementation. Mixing in "look at this cool plot!" demos would dilute that.

3. **nbdev convention** — Having a separate `index.nbs` or tutorial notebook that showcases usage is pretty standard. You can set `#| default_exp false` at the top (or just omit the `default_exp` directive entirely) so nothing gets exported.

4. **Documentation value** — A demo notebook can become part of your docs/README, showing "here's what you can do in 30 seconds."

One small consideration: if the demos are *also* serving as tests (e.g., "does plotting actually work?"), you might want a couple of minimal smoke-test cells in `core.nbs` too. But the fancy demos belong elsewhere.

So: **yes, make a separate `demo.nbs`** (or `tutorial.nbs` or `index.nbs` — whatever feels right for how you want it to appear in docs).

In [ ]:
#| hide

#import nbdev; nbdev.nbdev_export()
!nbdev_export --procs scrub_magics

Oh, there already is a index notebook, but that typically gets mapped immediately to the README. Do I want the demo stuff to be in the read me. I guess I could.... hmmm

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yeah, it's a judgment call! A few options:

1. **Put demos in `index.nbs`** — Your README *does* become a nice showcase of what the library can do. People landing on your repo immediately see "oh cool, progress bars and plots work remotely." That's compelling! Many popular libraries do this.

2. **Keep `index.nbs` minimal, separate `demo.nbs`** — If you want the README to be a quick "what is this / how to install / basic usage" and keep the flashier demos elsewhere. The demo notebook would still be browsable in the repo or linked from the README.

3. **Both** — A *taste* in `index.nbs` (one quick example), with "see `demo.nbs` for more examples" linking to the full showcase.

What's your instinct on how verbose you want your README to be? Some folks like a tight README that gets to the point; others like a "wow factor" landing page with visuals.

I think I'll keep the readme pretty minimal. Some basic instructions for setting it up. But then I will say see the demo.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

That sounds like a solid plan. Clean README with setup instructions, then point people to `demo.nbs` for the fun stuff. Best of both worlds — quick to scan for people who just want to install, but the visual demos are there for those who want to see it in action.

Go for it!